# Exploration Notebook to scrape several websites for job openings, store them in a database and build an own recommender for interesting jobs

In [1]:
from bs4 import BeautifulSoup
import undercover_request # custom module, install: pip install git+https://github.com/TimSchopf/undercover_request

In [2]:
#searched keywords: thesis
daimler_keywords = {'_': 1555512030476} 
#searched keywords: thesis
bosch_keywords = {"from":0,"size":9,"sort":[{"releasedDate":{"order":"desc"}}],"query":{"query_string":{"query":"thesis*","default_operator":"and"}}}

In [3]:
# Daimler uses an API to send the data to the client.
# The Daimler Carrer Website itself is just a HTML template without data.
# Data and template render client side and are sent seperately to the client.
# In this case we must find the API URL that returns the raw data (the Daimler API sends JSON).
# The API URL already conatins the searched keywords, we don't need to append the daimler_keywords additionally
daimler_api_url = 'https://global-jobboard-api.daimler.com/v3/search/%7B%22LanguageCode%22%3A%22DE%22%2C%22ScoreThreshold%22%3A80%2C%22SearchParameters%22%3A%7B%22MatchedObjectDescriptor%22%3A%5B%22PositionID%22%2C%22PositionTitle%22%2C%22PositionURI%22%2C%22LogoURI%22%2C%22OrganizationName%22%2C%22Organization%22%2C%22Organization.MemberCode%22%2C%22ParentOrganization%22%2C%22PositionLocation.CityName%22%2C%22PositionLocation.Longitude%22%2C%22PositionLocation.Latitude%22%2C%22PositionIndustry.Name%22%2C%22JobCategory.Name%22%2C%22JobCategory.Code%22%2C%22CareerLevel.Name%22%2C%22CareerLevel.Code%22%2C%22Facet%3AParentOrganization%22%2C%22Facet%3APositionLocation.CityName%22%2C%22Facet%3APositionLocation.CountryName%22%2C%22PublicationStartDate%22%2C%22ParentOrganizationGenesisID%22%5D%2C%22FirstItem%22%3A0%2C%22CountItem%22%3A1000000%7D%2C%22SearchCriteria%22%3A%5B%7B%22CriterionName%22%3A%22PublicationLanguage.Code%22%2C%22CriterionValue%22%3A%5B%22DE%22%5D%7D%2C%7B%22CriterionName%22%3A%22PublicationChannel.Code%22%2C%22CriterionValue%22%3A%5B%2212%22%5D%7D%2C%7B%22CriterionName%22%3A%22CareerLevel.Code%22%2C%22CriterionValue%22%3A%5B%2240%22%5D%7D%5D%7D?_=1555512030476'
daimler_req = undercover_request.request(daimler_api_url, request_type='get')

update user agent database
get proxy list
try proxies
SUCCESS [Number of attempts: 2] Proxy: {'ip': '81.30.216.147', 'port': '41258'} User Agent: Mozilla/5.0 (X11; OpenBSD i386) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.125 Safari/537.36


In [4]:
# Bosch also uses an API,returns JSON and expects JSON post parameter
# The Bosch API just delivers 9 results at a time, but more results can be loaded on the website via load more button
# Expectation: to receive all data the API needs post requests with updated "from" data (+9) until total hits<= updated "from" data
bosch_req = undercover_request.request('https://ro-dyn-backend.bosch.com/c-corpweb-tc/es/bosch-ro-backend*/ro_bosch_backend__job_posting/_search', request_type='post', json=bosch_keywords)

update user agent database
get proxy list
try proxies
SUCCESS [Number of attempts: 15] Proxy: {'ip': '176.196.195.170', 'port': '48129'} User Agent: Mozilla/5.0 (Macintosh; Intel Mac OS X 10_8_2) AppleWebKit/537.17 (KHTML, like Gecko) Chrome/24.0.1309.0 Safari/537.17


In [5]:
daimler_data = daimler_req.json()
bosch_data = bosch_req.json()

In [6]:
# to find where exactly in JSOn file the intersting data is stored, use a JSON viewer or the preview window in chrome inspector
i = 0
# job title 
title = bosch_data['hits']['hits'][i]['_source']['name']
# detailed job description
description = bosch_data['hits']['hits'][i]['_source']['jobAd']['sections']['jobDescription']['text']
# job qualifications
qualifications = bosch_data['hits']['hits'][i]['_source']['jobAd']['sections']['qualifications']['text']
# url to detailed job descrition and application possibility
applyURL = bosch_data['hits']['hits'][i]['_source']['applyUrl']
# release date of job offer
releaseDate = bosch_data['hits']['hits'][i]['_source']['releasedDate']